In [28]:
import torch.nn.functional as F
import torch
import torchvision
from matplotlib import pyplot as plt
from torchvision import utils
import numpy as np
import torch.nn as nn
import os
import pandas as pd
import PIL
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
writer = SummaryWriter("runs/STRATIFIED IMG-ONLY")

myseed = 0
torch.manual_seed(myseed)
np.random.seed(myseed)
generator=torch.Generator()
generator.manual_seed(myseed)

%matplotlib inline

In [29]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

cpu


In [30]:
data_path = "./"

In [31]:
images = os.listdir(f"{data_path}DATASET")
print(images)

['P_3_64.png', 'P_3_60.png', 'P_501.png', 'P_3_424.png', 'P_1_128.png', 'P_471.png', 'P_716.png', 'P_669.png', 'P_217.png', 'P_486.png', 'P_1_163.png', 'P_3_106.png', 'P_3_36.png', 'P_8.png', 'P_59.png', 'P_3_382.png', 'P_1_38.png', 'P_3_468.png', 'P_3_154.png', 'P_628.png', 'P_663.png', 'P_327.png', 'P_3_169.png', 'P_790.png', 'P_3_374.png', 'P_143.png', 'P_3_102.png', 'P_733.png', 'P_3_118.png', 'P_19.png', 'P_3_407.png', 'P_1_59.png', 'P_1_12.png', 'P_113.png', 'P_341.png', 'P_1_148.png', 'P_1_80.png', 'P_424.png', 'P_3_311.png', 'P_1_49.png', 'P_3_98.png', 'P_602.png', 'P_181.png', 'P_3_358.png', 'P_191.png', 'P_1_84.png', 'P_659.png', 'P_620.png', 'P_571.png', 'P_3_125.png', 'P_2_65.png', 'P_810.png', 'P_3_415.png', 'P_667.png', 'P_1_99.png', 'P_584.png', 'P_608.png', 'P_718.png', 'P_2_14.png', 'P_664.png', 'P_442.png', 'P_3_325.png', 'P_817.png', 'P_1_106.png', 'P_252.png', 'P_2_94.png', 'P_3_72.png', 'P_3_166.png', 'P_110.png', 'P_3_145.png', 'P_37.png', 'P_137.png', 'P_547.png'

In [32]:
len(images)

1589

In [33]:
df = pd.read_excel("trainANDtest.xls")

In [34]:
df

,Unnamed: 0,Row_number,ImageFile,Hospital,Age,Sex,PositivityAtAdmission,Temp_C,DaysFever,Cough,...,pH,HighBloodPressure,Diabetes,Dementia,BPCO,Cancer,ChronicKidneyDisease,RespiratoryFailure,Prognosis,Death
0,0,82.0,P_102.png,1,72,1,1,37.3,3,1,...,7.46,0,0,0,0,1,0,0,1,0
1,1,228.0,P_131.png,4,35,0,1,39.3,3,1,...,7.46,1,0,0,0,0,0,0,0,0
2,2,229.0,P_132.png,4,57,0,1,37.0,3,0,...,7.42,0,0,0,0,0,0,0,0,0
3,3,96.0,P_117.png,1,39,1,1,37.2,2,1,...,7.38,0,0,0,0,0,0,0,0,0
4,4,112.0,P_16.png,1,44,0,1,38.0,3,1,...,7.47,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584,481,1474.0,P_3_432.png,6,84,1,1,37.0,2,0,...,7.46,1,0,1,0,0,0,0,0,0
1585,482,1228.0,P_3_105.png,6,51,0,1,38.0,3,0,...,7.44,0,1,0,0,0,0,0,1,0
1586,483,1466.0,P_3_233.png,6,70,1,1,36.4,2,1,...,7.50,1,0,0,0,0,0,0,0,0
1587,484,1210.0,P_3_414.png,6,90,0,1,36.2,4,0,...,7.42,0,1,0,0,0,0,0,0,0


In [35]:
del df["Row_number"]
del df["Unnamed: 0"]

In [36]:
print("The total number of patients is", len(df))

The total number of patients is 1589


In [37]:
print("Classes are:")
print(df["Prognosis"].value_counts())

Classes are:
Prognosis
0    841
1    748
Name: count, dtype: int64


In [38]:
df.columns

Index(['ImageFile', 'Hospital', 'Age', 'Sex', 'PositivityAtAdmission',
       'Temp_C', 'DaysFever', 'Cough', 'DifficultyInBreathing', 'WBC', 'RBC',
       'CRP', 'Glucose', 'LDH', 'INR', 'PaO2', 'PaCO2', 'pH',
       'HighBloodPressure', 'Diabetes', 'Dementia', 'BPCO', 'Cancer',
       'ChronicKidneyDisease', 'RespiratoryFailure', 'Prognosis', 'Death'],
      dtype='object')

In [39]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, indices, image_dir, transform=None):
        self.image_dir = image_dir
        self.indices = indices
        self.transform = transform

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tabular = self.indices.iloc[idx, 0:]

        y = tabular["Prognosis"]

        image = PIL.Image.open(f"{self.image_dir}/{tabular['ImageFile']}")
        image = image.convert('L')
        image = np.array(image)
        #image = image[..., :3]

        image = transforms.functional.to_tensor(image)

        tabular = tabular[['Age', 'Sex', 'PositivityAtAdmission',
       'Temp_C', 'DaysFever', 'Cough', 'DifficultyInBreathing', 'WBC', 'RBC',
       'CRP', 'Glucose', 'LDH', 'INR', 'PaO2', 'PaCO2', 'pH',
       'HighBloodPressure', 'Diabetes', 'Dementia', 'BPCO', 'Cancer',
       'ChronicKidneyDisease', 'RespiratoryFailure']]
        tabular = tabular.tolist()
        tabular = torch.FloatTensor(tabular)
        
        if self.transform:
            image = self.transform(image)

        return image, tabular, y
    

In [40]:
from torchvision import transforms as T
my_transform = T.Compose([T.Resize((256,256)),
                          T.RandomApply(
                            [T.RandomHorizontalFlip(),
                             T.RandomCrop(256, padding=4)],
                            p=.5
                          )])

In [41]:
from sklearn.model_selection import train_test_split

tv_idx, test_idx = train_test_split(np.arange(len(df["Prognosis"])), test_size=0.2, shuffle=True, stratify=df["Prognosis"])

In [42]:
train_val_df = df.iloc[tv_idx]
train_val = ImageDataset(indices=train_val_df, image_dir=f"{data_path}DATASET", transform=my_transform)

In [43]:
len(train_val)

1271

In [44]:
test_df = df.iloc[test_idx]
test_set = ImageDataset(indices=test_df, image_dir=f"{data_path}DATASET", transform=my_transform)

len(test_set)

318

In [45]:
i = 3
print(f"{train_val[i][0].size()}, label = {train_val[i][1]}")

torch.Size([1, 256, 256]), label = tensor([ 73.0000,   1.0000,   1.0000,  37.4000,   2.0000,   1.0000,   0.0000,
          4.2000,   4.6500,   1.9200, 134.0000, 223.0000,   0.9600,  73.0000,
         31.0000,   7.4400,   1.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000])


In [46]:
torch.cuda.is_available()

False

In [47]:
dev = torch.device('cuda')
print(dev)
# dev = torch.device("cpu")
# print(dev)

cuda


# 1. SOLO IMMAGINI

In [48]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
               
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, in_channels=1, num_classes=2):
        super(ResNet, self).__init__()
        torch.manual_seed(myseed)
        self.in_planes = 64

        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(in_channels, num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], in_channels=in_channels, num_classes=num_classes)

model = ResNet18(1,1)
print(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [49]:
# Define an optimizier
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
# Define a loss 
criterion = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = 0.01, epochs=5, steps_per_epoch=350)


In [50]:
def train(net, loaders, optimizer, criterion, epochs=500, dev=dev, save_param=False, model_name="only-images"):
    # Ensure history is initialized at the top level
    history_loss = {"train": [], "val": [], "test": []}
    history_accuracy = {"train": [], "val": [], "test": []}
    torch.manual_seed(myseed)
    best_val_accuracy = 0

    try:
        net = net.to(dev)
        # Process each epoch
        for epoch in range(epochs):
            # Initialize epoch variables
            sum_loss = {"train": 0, "val": 0, "test": 0}
            sum_accuracy = {"train": 0, "val": 0, "test": 0}

            # Process each split
            for split in ["train", "val", "test"]:
                if split == "train":
                    net.train()
                else:
                    net.eval()

                # Process each batch
                for (input, tabular, labels) in loaders[split]:
                    input, labels = input.to(dev), labels.to(dev)

                    # Reset gradients for training split
                    if split == "train":
                        optimizer.zero_grad()

                    # Compute output
                    pred = net(input)
                    labels = labels.unsqueeze(1).float()  # Adjust labels for BCEWithLogitsLoss
                    loss = criterion(pred, labels)

                    # Update loss
                    sum_loss[split] += loss.item()

                    # Perform backward pass and optimization for training split
                    if split == "train":
                        loss.backward()
                        optimizer.step()

                    # Compute accuracy
                    pred_labels = (pred >= 0).long()  # Binarize predictions
                    batch_accuracy = (pred_labels == labels).sum().item() / input.size(0)
                    sum_accuracy[split] += batch_accuracy

                # Update the learning rate scheduler for training split
                if split == "train":
                    scheduler.step()

            # Compute epoch loss and accuracy
            epoch_loss = {split: sum_loss[split] / len(loaders[split]) for split in ["train", "val", "test"]}
            epoch_accuracy = {split: sum_accuracy[split] / len(loaders[split]) for split in ["train", "val", "test"]}

            # Save parameters if validation accuracy improves
            if save_param and epoch_accuracy["val"] > best_val_accuracy:
                torch.save(net.state_dict(), f"{model_name}_best_val.pth")
                best_val_accuracy = epoch_accuracy["val"]

            # Update history
            for split in ["train", "val", "test"]:
                history_loss[split].append(epoch_loss[split])
                history_accuracy[split].append(epoch_accuracy[split])

            # Log to TensorBoard
            writer.add_scalar("Train Loss", epoch_loss["train"], epoch)
            writer.add_scalar("Valid Loss", epoch_loss["val"], epoch)
            writer.add_scalar("Test Loss", epoch_loss["test"], epoch)
            writer.add_scalar("Train Accuracy", epoch_accuracy["train"], epoch)
            writer.add_scalar("Valid Accuracy", epoch_accuracy["val"], epoch)
            writer.add_scalar("Test Accuracy", epoch_accuracy["test"], epoch)

            # Print epoch info
            print(f"Epoch {epoch + 1}:",
                  f"TrL={epoch_loss['train']:.4f},",
                  f"TrA={epoch_accuracy['train']:.4f},",
                  f"VL={epoch_loss['val']:.4f},",
                  f"VA={epoch_accuracy['val']:.4f},",
                  f"TeL={epoch_loss['test']:.4f},",
                  f"TeA={epoch_accuracy['test']:.4f},",
                  f"LR={optimizer.param_groups[0]['lr']:.5f}")

    except KeyboardInterrupt:
        print("Training interrupted. Finalizing plots.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Ensure history variables are safe to plot
        if any(len(history_loss[split]) > 0 for split in ["train", "val", "test"]):
            plt.title("Loss")
            for split in ["train", "val", "test"]:
                plt.plot(history_loss[split], label=split)
            plt.legend()
            plt.show()

        if any(len(history_accuracy[split]) > 0 for split in ["train", "val", "test"]):
            plt.title("Accuracy")
            for split in ["train", "val", "test"]:
                plt.plot(history_accuracy[split], label=split)
            plt.legend()
            plt.show()

In [51]:
def reset_weights(m):

    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

In [52]:
test_loader  = DataLoader(test_set,  batch_size=8, num_workers=2, drop_last=False, shuffle=False, generator=generator)

In [53]:
tv_labels = train_val_df['Prognosis'].tolist()

In [54]:
# Stratified K-Fold Cross-Validation Loop
epochs = 500
skf = StratifiedKFold(n_splits=5)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_val, tv_labels)):
    writer = SummaryWriter("runs/STRATIFIED IMG-ONLY", filename_suffix=f"_F{fold}_E{epochs}_")
    print('------------fold no---------{}----------------------'.format(fold))

    train_df = df.iloc[train_idx]
    train_set = ImageDataset(indices=train_df, image_dir=f"{data_path}DATASET", transform=my_transform)

    val_df = df.iloc[val_idx]
    val_set = ImageDataset(indices=val_df, image_dir=f"{data_path}DATASET", transform=my_transform)

    print("Total samples in each hospital:")
    for hospital_id in range(1, 7):
        print(f"Hospital {hospital_id}: {len(train_df[train_df['Hospital'] == hospital_id])}")

    train_loader = DataLoader(train_set, batch_size=8, num_workers=2, drop_last=True, shuffle=True, generator=generator)
    val_loader = DataLoader(val_set, batch_size=8, num_workers=2, drop_last=False, shuffle=False, generator=generator)

    # Define dictionary of loaders
    loaders = {"train": train_loader,
               "val": val_loader,
               "test": test_loader}

    # Define optimizer, loss, and scheduler
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    criterion = nn.BCEWithLogitsLoss()
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, epochs=5, steps_per_epoch=len(train_loader))

    # Train model
    train(model, loaders, optimizer, criterion, epochs=epochs, dev=dev)
    writer.flush()
    writer.close()

    # Reset model weights
    model.apply(reset_weights)


------------fold no---------0----------------------
Total samples in each hospital:
Hospital 1: 99
Hospital 2: 76
Hospital 3: 120
Hospital 4: 115
Hospital 5: 66
Hospital 6: 540
An unexpected error occurred: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx


------------fold no---------1----------------------
Total samples in each hospital:
Hospital 1: 85
Hospital 2: 81
Hospital 3: 124
Hospital 4: 95
Hospital 5: 86
Hospital 6: 546
An unexpected error occurred: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx
------------fold no---------2----------------------
Total samples in each hospital:
Hospital 1: 84
Hospital 2: 79
Hospital 3: 127
Hospital 4: 99
Hospital 5: 81
Hospital 6: 547
An unexpected error occurred: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx
------------fold no---------3----------------------
Total samples in each hospital:
Hospital 1: 98
Hospital 2: 77
Hospital 3: 100
Hospital 4: 108
Hospital 5: 70
Hospital 6: 564
An unexpected error occurred: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU a